In [1]:
import sys
import os
import imp
import pickle as pickle
import numpy as np

from optparse import OptionParser
from demos.diffusion import ContaminantTransportModel
from collections import Iterable
from paper.examples.diffusion_common import *
from vuq import *

In [2]:
def make_model():
    """
    Make the diffusion model.
    """
    # The number of dimensions for this particular problem
    num_dim = 3

    # The log prior
    log_prior = PDFCollection([UniformND(2),
                               MultivariateNormal([[-1.]])])
    y = load_diffusion_data()
    solver = ContaminantTransportModel()
    log_like = IsotropicGaussianLikelihood(y, solver)
    log_p = Joint(log_like, log_prior)

    return locals()

In [3]:
def parse_expr_callback(option, opt, value, parser):
    if value is not None:
        setattr(parser.values, option.dest, eval(value))


def parse_list_callback(option, opt, value, parser):
    if value is not None:
        value = [float(x) for x in value.split(',')]
        if len(value) == 0:
            value = value[0] 
    setattr(parser.values, option.dest, value)


def convert_to_list(l, n):
    if not isinstance(l, Iterable):
        l = [l] * n
    return l


def initialize_bounds(l, u, n):
    l = convert_to_list(l, n)
    u = convert_to_list(u, n)
    b = tuple((l[i], u[i]) for i in range(n))
    return b


def main(options):
    """
    The main function.
    """
    # Load the model
    model = options.model#initialize_native_model(options.model)
    # Get the target
    log_p = model['log_p']
    # Get the prior
    log_prior = model['log_prior']
    # Construct the initial approximation
    comp = [MultivariateNormal(log_prior.sample().flatten())
            for i in range(options.num_comp)]
    log_q = MixtureOfMultivariateNormals(comp)
    if options.mu_init is not None:
        log_q.mu = options.mu_init
        print (str(log_q))
    # The entropy approximation
    entropy = eval(options.entropy_approximation + '()')
    # The expectation functional
    expectation_functional = eval(options.expectation_functional + '(log_p)')
    # The Evidence-Lower-BOund
    elbo = EvidenceLowerBound(entropy, expectation_functional)
    # The optimizer
    optimizer = Optimizer(elbo)
    # The upper and lower bounds for everything
    mu_bounds = initialize_bounds(options.mu_lower_bound, options.mu_upper_bound, log_q.num_dim)
    C_bounds = initialize_bounds(options.C_lower_bound, options.C_upper_bound, log_q.num_dim)
    print ('mu_bounds', mu_bounds)
    print ('C_bounds', C_bounds)
    # The output file
    output_file = options.output
    if output_file is None:
        output_file = os.path.abspath('Ave_Maria' + '_'
                                      + 'num_comp=' + str(options.num_comp) + '.pcl')
    # Delete the output file if it exists and you want to force calculations
    if os.path.exists(output_file) and options.force:
        print ('-', output_file, 'exists')
        print ('- I am removing it')
        os.remove(output_file)
    # If the file exists at this point, then this is not a forced calculation
    # and we should just load it
    if os.path.exists(output_file):
        print ('- I am not repeating the calculations')
        with open(output_file, 'rb') as fd:
            results = pickle.load(fd)
            L = results['L']
            log_q = results['log_q']
            nfev = results['nfev']
    else:
        # Do the optimization
        L, nfev = optimizer.optimize(log_q,
                                     tol=options.tol,
                                     max_it=options.max_it,
                                     mu_bounds=mu_bounds,
                                     C_bounds=C_bounds,
                                     full_mu=options.optimize_full_mu)
        print (str(elbo))
        # Save the results
        results = {}
        results['L'] = L
        results['log_q'] = log_q
        results['nfev'] = nfev
        with open(output_file, 'wb') as fd:
            pickle.dump(results, fd, protocol=pickle.HIGHEST_PROTOCOL)
    # Print a salary of the statistics
    w = log_q.w
    mu = log_q.mu
    C = log_q.C
    c = np.vstack([np.diag(C[i, :, :]) for i in range(log_q.num_comp)])
    x_m = np.mean(w * mu, axis=0)
    # The variance of the mixture can only be approximated via sampling...
    samples = log_q.sample(options.std_samples)
    x_s = np.std(samples, axis=0)
    x_05 = np.percentile(samples, 5, axis=0)
    x_95 = np.percentile(samples, 95, axis=0)
    print ('{0:10s} {1:10s} {2:10s}'.format('Parameter', 'Mean', 'Std.'))
    print ('-' * 32)
    for i in range(log_q.num_dim):
        print ('{0:10s} {1:4.6f} +-{2:2.6f}'.format('x_' + str(i+1), x_m[i], 1.96 * x_s[i]))
    for i in range(log_q.num_dim):
        print ('(%1.3f, %1.3f)' % (x_05[i], x_95[i]))
    print ('Number of evaluations:', nfev)

In [4]:
def init_mpi(use_mpi):
    """
    Initialize mpi (if requested).
    """
    if use_mpi is not None:
        import mpi4py.MPI as mpi
        comm = mpi.COMM_WORLD
        rank = comm.Get_rank()
        size = comm.Get_size()
        return mpi, comm, rank, size
    else:
        return None, None, 0, 1


def get_rank_size(comm):
    """
    Get the rank and size from a communicator.
    """
    if comm is None:
        return 0, 1
    rank = comm.Get_rank()
    size = comm.Get_size()
    return rank, size


def print_once(msg, comm=None):
    """
    Use to print a message once when using mpi.
    """
    if comm is None:
        sys.stdout.write(msg)
    else:
        rank = comm.Get_rank()
        if rank == 0:
            sys.stdout.write(msg)
            sys.stdout.flush()
        comm.barrier()


def initialize_model(model_file, model_name='Model', comm=None):
    """
    Initialize the ``model_file`` which should be a proper python module.
    """
    rank, size = get_rank_size(comm)
    print_once('Initializing the %s.\n' % model_name, comm=comm)
    print_once('-------------------------\n', comm=comm)
    for i in range(size):
        if rank == i:
            try:
                model = imp.load_source('', model_file)
            except Exception as e:
                signal_fatal_error('I couldn\'t load the %s.\n' % model_name,
                                   comm=comm, e=e)
            print_rank(i, 'initialized %s.\n' % model_name)
        mpi_wait(comm)
    print_once('Done.\n'
               '-------------------------\n', comm=comm)
    return model


def initialize_pymc_model(model_file, model_name='PyMC Model', comm=None):
    """
    Initialize a PyMC model.
    """
    return initialize_model(model_file, model_name=model_name, comm=comm).make_model()


def initialize_native_model(model_file, model_name='Native Model', comm=None):
    """
    Initialize a native model.
    """
    return initialize_model(model_file, model_name=model_name, comm=comm).make_model()


def signal_fatal_error(err, err_code=1, comm=None, e=None):
    """
    Signal a fatal error. The program will exit.
    """
    sys.stderr.write(err)
    if comm is None:
        if e is not None:
            print (e)
        sys.exit(1)
    else:
        if comm.Get_rank() == 0:
            print (e)
        comm.Abort(1)


def print_rank(i, msg):
    """
    Use to print information from a particular rank.
    This is an mpi only function.
    """
    sys.stdout.write('rank %d: %s' % (i, msg))
    sys.stdout.flush()


def mpi_wait(comm):
    """
    Wait if comm is not ``None``.
    """
    if comm is not None:
        comm.barrier()


def get_running_statistics(data):
    """
    Returns the running mean and standard deviation of the data.
    """
    x_m = np.cumsum(data, axis=0) / np.arange(1, data.shape[0] + 1)[:, None]
    x2_m = np.cumsum(data ** 2., axis=0) / np.arange(1, data.shape[0] + 1)[:, None]
    x_s = np.sqrt(x2_m - x_m ** 2.)
    return x_m, x_s


In [5]:
parser = OptionParser()
parser.add_option('-m', '--model', metavar='FILE', dest='model', default=make_model(),
                  help='specify the file containing the native model')
parser.add_option('--entropy-approximation', type=str, dest='entropy_approximation',
                  default='FirstOrderEntropyApproximation',
                  help='specify the entropy approximation to be used')
parser.add_option('--expectation-functional', type=str, dest='expectation_functional',
                  default='ThirdOrderExpectationFunctional',
                  help='specify the expectation functional to be used')
parser.add_option('--num-comp', type=int, dest='num_comp', default=1,
                  help='specify the number of components that you want to use')
parser.add_option('--mu-lower-bound', type=str, dest='mu_lower_bound', action='callback',
                  callback=parse_expr_callback,
                  help='specify the lower bound for mu')
parser.add_option('--mu-upper-bound', type=str, dest='mu_upper_bound', default=[1,1]+[None], action='callback',
                  callback=parse_expr_callback, 
                  help='specify the upper bound for mu') 
parser.add_option('--C-lower-bound', type=str, dest='C_lower_bound', action='callback',
                  callback=parse_expr_callback, default=1e-6,
                  help='specify the lower bound for C')
parser.add_option('--C-upper-bound', type=str, dest='C_upper_bound', action='callback',
                  callback=parse_expr_callback, default=[10]*3,
                  help='specify the upper bound for C')
parser.add_option('--max-it', type=int, dest='max_it', default=100,
                  help='specify the maximum number of iterations')
parser.add_option('--tol', type=float, dest='tol', default=1e-2,
                  help='specify the tolerance')
parser.add_option('--std-samples', type=int, dest='std_samples', default=10000,
                  help='specify the number of samples used in the estimation of the std')
parser.add_option('--optimize-full-mu', action='store_true', dest='optimize_full_mu',
                  help='optimize all the mus simultaneously')
parser.add_option('-f', '--force', action='store_true', dest='force',
                  help='force the calculation')
parser.add_option('-o', '--output', metavar='FILE', dest='output',
                  help='specify the output file')
parser.add_option('--mu-init', type=str, dest='mu_init', action='callback',
                  callback=parse_expr_callback,
                  help='specify the initial point for mu num_comp x num_dim')
options, args = parser.parse_args()
main(options)

mu_bounds ((None, 1), (None, 1), (None, None))
C_bounds ((1e-06, 10), (1e-06, 10), (1e-06, 10))
- C:\Users\Fedor\Documents\GitHub\variational-reformulation-of-inverse-problems\Ave_Maria_num_comp=1.pcl exists
- I am removing it
mu [ 0.2780609   0.88626085 -0.7135421 ]
mu [ 0.31512841  0.10081994 -0.09571641]
mu [ 0.40172526  0.38254053 -0.43562306]
mu [ 0.19992994  0.01028217 -1.22746813]
mu [ 0.36356808  0.31215074 -0.58535188]
mu [ 0.24215179  0.08817019 -1.06178954]
mu [ 0.21664439  0.04111586 -1.16188059]
mu [-0.73829913 -0.812229   -6.98233096]
mu [-0.06865844 -0.21383294 -2.90082215]
mu [ 0.18099519  0.00925946 -1.37916507]
mu [ 0.20106354  0.02719269 -1.25684705]
mu [-0.52715599 -0.03743718 -6.7765212 ]
mu [-0.01483002  0.00803202 -2.89325208]
mu [ 0.13012166  0.02089656 -1.79456409]
mu [ 0.17033414  0.01973866 -2.75690703]
mu [ 0.15072535  0.02030328 -2.28764034]
mu [ 0.4760353   0.13453666 -4.30424755]
mu [ 0.23206939  0.04886744 -2.79189477]
mu [ 0.18767467  0.03327813 -2.5166